In [2]:
#!<python -m pip install --upgrade --user ortools

In [3]:
from ortools.graph import pywrapgraph

#https://developers.google.com/optimization/flow/mincostflow
#https://towardsdatascience.com/operations-research-in-r-transportation-problem-1df59961b2ad

def solve_transportation_problem(start_nodes, end_nodes, capacities, unit_costs, supplies ):
    """MinCostFlow simple interface example."""
    # Instantiate a SimpleMinCostFlow solver.
    min_cost_flow = pywrapgraph.SimpleMinCostFlow()

    # Add each arc.
    for arc in zip(start_nodes, end_nodes, capacities, unit_costs):
        min_cost_flow.AddArcWithCapacityAndUnitCost(arc[0], arc[1], arc[2],
                                                    arc[3])

    # Add node supply.
    for count, supply in enumerate(supplies):
        min_cost_flow.SetNodeSupply(count, supply)

    # Find the min cost flow.
    status = min_cost_flow.Solve()

    if status != min_cost_flow.OPTIMAL:
        print('There was an issue with the min cost flow input.')
        print(f'Status: {status}')
        exit(1)
    print('Minimum cost: ', min_cost_flow.OptimalCost())
    print('')
    print(' Arc   Flow / Capacity  Cost')
    for i in range(min_cost_flow.NumArcs()):
        cost = min_cost_flow.Flow(i) * min_cost_flow.UnitCost(i)
        print('%1s -> %1s    %3s   / %3s   %3s' %
              (min_cost_flow.Tail(i), min_cost_flow.Head(i),
               min_cost_flow.Flow(i), min_cost_flow.Capacity(i), cost))
    

<img src="img/Google_Transportation_Example.png">

In [8]:
# Google Example

# Define four parallel arrays: sources, destinations, capacities,
# and unit costs between each pair. For instance, the arc from node 0
# to node 1 has a capacity of 15.
start_nodes = [0, 0, 1, 1, 1, 2, 2, 3, 4]
end_nodes = [1, 2, 2, 3, 4, 3, 4, 4, 2]
capacities = [15, 8, 20, 4, 10, 15, 4, 20, 5]
unit_costs = [4, 4, 2, 2, 6, 1, 3, 2, 3]

# Define an array of supplies at each node.
supplies = [20, 0, 0, -5, -15]

In [9]:
solve_transportation_problem(start_nodes, end_nodes, capacities, unit_costs, supplies)

Minimum cost:  150

 Arc   Flow / Capacity  Cost
0 -> 1     12   /  15    48
0 -> 2      8   /   8    32
1 -> 2      8   /  20    16
1 -> 3      4   /   4     8
1 -> 4      0   /  10     0
2 -> 3     12   /  15    12
2 -> 4      4   /   4    12
3 -> 4     11   /  20    22
4 -> 2      0   /   5     0


<img src="img/Transportation_Problem.png">

In [6]:
# Supply is higer than demand ( 1600 > 1400), then create a dummy client 
# which will be node 6. 

# There is not capacity restriction. Then assume a large number por capacity c.
c = 10**6

start_nodes = [0, 0, 0, 1, 1, 1, 2, 2, 2] + [0, 1, 2]
end_nodes = [3, 4, 5, 3, 4, 5, 3, 4, 5] + [6, 6, 6]
capacities = [c, c, c, c, c, c, c, c, c] + [c, c, c]
unit_costs = [4, 7, 6, 5, 5, 5, 9, 5, 8] + [0, 0, 0]

# Define an array of supplies at each node.
supplies = [400, 700, 500, -600, -500, -300] + [-200]

In [7]:
solve_transportation_problem(start_nodes, end_nodes, capacities, unit_costs, supplies)

Minimum cost:  6600

 Arc   Flow / Capacity  Cost
0 -> 3    400   / 1000000   1600
0 -> 4      0   / 1000000     0
0 -> 5      0   / 1000000     0
1 -> 3    200   / 1000000   1000
1 -> 4      0   / 1000000     0
1 -> 5    300   / 1000000   1500
2 -> 3      0   / 1000000     0
2 -> 4    500   / 1000000   2500
2 -> 5      0   / 1000000     0
0 -> 6      0   / 1000000     0
1 -> 6    200   / 1000000     0
2 -> 6      0   / 1000000     0


# 5 Suppliers, 25 Clients Problem

In [40]:
import pandas as pd


#pd.read_excel
df_cap = pd.read_excel('docs/io.xlsx', 
            sheet_name = 'Capacities', 
            usecols = [0,1,2], 
             dtype = {
                        'NodeIndex': str, 
                        'NodeName': str, 
                        'Capacity': float})

# Load Nodes connections
df_conn = pd.read_excel('docs/io.xlsx', 
                    sheet_name = 'Connections', usecols = [0,1,2,3], 
                    dtype = {
                        'StartNode': str, 
                        'EndNode': str, 
                        'Distance': float,
                        'Cost': float})

In [43]:
# Eliminate Supplier 6
df_cap = df_cap[df_cap['NodeName'] != 'Prov 6'].copy(deep = True)

# Reindex Nodes
df_cap['NodeIndex'] = df_cap.index

# Drop na's
df_cap.dropna(inplace = True)

In [45]:
df_conn.head()

# Eliminate Supplier 6
df_conn = df_conn[(df_conn['StartNode'] != 'Prov 6') or (df_conn['NextNode'] != 'Prov 6')].copy(deep = True)

,StartNode,EndNode,Distance,Cost
0,Prov 1,Prov 2,0.7,1.0
1,Prov 1,Prov 3,1.3,1.0
2,Prov 1,Prov 4,1.5,1.0
3,Prov 1,Prov 5,2.5,1.0
4,Prov 1,Cliente 1,5.2,1.0


In [33]:
node_mapper = dict(zip(df_cap['NodeName'], df_cap['NodeIndex']))

In [35]:
node_mapper

{'Prov 1': '0',
 'Prov 2': '1',
 'Prov 3': '2',
 'Prov 4': '3',
 'Prov 5': '4',
 'Prov 6': '5',
 'Cliente 1': '6',
 'Cliente 2': '7',
 'Cliente 3': '8',
 'Cliente 4': '9',
 'Cliente 5': '10',
 'Cliente 6': '11',
 'Cliente 7': '12',
 'Cliente 8': '13',
 'Cliente 9': '14',
 'Cliente 10': '15',
 'Cliente 11': '16',
 'Cliente 12': '17',
 'Cliente 13': '18',
 'Cliente 14': '19',
 'Cliente 15': '20',
 'Cliente 16': '21',
 'Cliente 17': '22',
 'Cliente 18': '23',
 'Cliente 19': '24',
 'Cliente 20': '25',
 'Cliente 21': '26',
 'Cliente 22': '27',
 'Cliente 23': '28',
 'Cliente 24': '29',
 'Cliente 25': '30',
 'Cliente 26': '31',
 nan: nan}

In [29]:
df_cap.head()

,NodeIndex,NodeName,Capacity
0,0,Prov 1,430.0
1,1,Prov 2,550.0
2,2,Prov 3,465.0
3,3,Prov 4,467.0
4,4,Prov 5,564.0


data